In [3]:
import pandas as pd #pd.read_csv
import numpy as np #linear algebra
import re
import itertools
import matplotlib.pyplot as plt #plotting
from sklearn.feature_extraction.text import TfidfVectorizer #turns each word into a unique word vector
from sklearn.metrics.pairwise import linear_kernel #applies cosine similarity
from fuzzywuzzy import fuzz #!pip install fuzzywuzzy if you dont own already
import gradio as gr #!pip install gradio, interface

In [4]:
fn = 'C:\\Users\\antho\\Desktop\\SteamCleanData\\steam.csv'
df = pd.read_csv(fn, encoding ='utf-8')

In [5]:
df

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,Adventure;Indie;Casual,7,3,0,0,0,0-20000,2.09
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,Action;Indie;Adventure,0,8,1,0,0,0-20000,1.69
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,Action;Indie;Casual,24,0,1,0,0,0-20000,3.99
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,Indie;Casual;Adventure,0,2,0,0,0,0-20000,5.19


In [6]:
df.shape

(27075, 18)

In [7]:
#We need to make a function that'll extract the years from the release date, so we'll use extract_year
def extract_year(date):
   year = date[:4]
#Not all games will have a year though, so we will have to establish a way for those games to return a value that isnt a year as well
   if year.isnumeric():
      return int(year)
   else:
      return np.nan

df['year'] = df['release_date'].apply(extract_year)
df.head()

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,steamspy_tags,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,year
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,124534,3339,17612,317,10000000-20000000,7.19,2000
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,3318,633,277,62,5000000-10000000,3.99,1999
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,FPS;World War II;Multiplayer,0,3416,398,187,34,5000000-10000000,3.99,2003
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,Action;FPS;Multiplayer,0,1273,267,258,184,5000000-10000000,3.99,2001
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,FPS;Action;Sci-fi,0,5250,288,624,415,5000000-10000000,3.99,1999


In [8]:
#Moving on, I think a good thing we can gauge is by making a scoring system. 
#First, we'll be working on making a scoring column

#Create score column
def create_score(row):
  positive_count = row['positive_ratings']  
  negative_count = row['negative_ratings']
  total_count = positive_count + negative_count
  average = positive_count / total_count
  return round(average, 2)
#Here we make a total ratings column by adding both the positive and negative columns together.
def total_ratings(row):
  positive_count = row['positive_ratings']  
  negative_count = row['negative_ratings']
  total_count = positive_count + negative_count
  return total_count

df['total_ratings'] = df.apply(total_ratings, axis=1)
df['score'] = df.apply(create_score, axis=1)


In [9]:
df

,appid,name,release_date,english,developer,publisher,platforms,required_age,categories,genres,...,achievements,positive_ratings,negative_ratings,average_playtime,median_playtime,owners,price,year,total_ratings,score
0,10,Counter-Strike,2000-11-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,124534,3339,17612,317,10000000-20000000,7.19,2000,127873,0.97
1,20,Team Fortress Classic,1999-04-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,3318,633,277,62,5000000-10000000,3.99,1999,3951,0.84
2,30,Day of Defeat,2003-05-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Valve Anti-Cheat enabled,Action,...,0,3416,398,187,34,5000000-10000000,3.99,2003,3814,0.90
3,40,Deathmatch Classic,2001-06-01,1,Valve,Valve,windows;mac;linux,0,Multi-player;Online Multi-Player;Local Multi-P...,Action,...,0,1273,267,258,184,5000000-10000000,3.99,2001,1540,0.83
4,50,Half-Life: Opposing Force,1999-11-01,1,Gearbox Software,Valve,windows;mac;linux,0,Single-player;Multi-player;Valve Anti-Cheat en...,Action,...,0,5250,288,624,415,5000000-10000000,3.99,1999,5538,0.95
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
27070,1065230,Room of Pandora,2019-04-24,1,SHEN JIAWEI,SHEN JIAWEI,windows,0,Single-player;Steam Achievements,Adventure;Casual;Indie,...,7,3,0,0,0,0-20000,2.09,2019,3,1.00
27071,1065570,Cyber Gun,2019-04-23,1,Semyon Maximov,BekkerDev Studio,windows,0,Single-player,Action;Adventure;Indie,...,0,8,1,0,0,0-20000,1.69,2019,9,0.89
27072,1065650,Super Star Blast,2019-04-24,1,EntwicklerX,EntwicklerX,windows,0,Single-player;Multi-player;Co-op;Shared/Split ...,Action;Casual;Indie,...,24,0,1,0,0,0-20000,3.99,2019,1,0.00
27073,1066700,New Yankee 7: Deer Hunters,2019-04-17,1,Yustas Game Studio,Alawar Entertainment,windows;mac,0,Single-player;Steam Cloud,Adventure;Casual;Indie,...,0,2,0,0,0,0-20000,5.19,2019,2,1.00


In [10]:
df[['name', 'total_ratings', 'score', 'price']]

,name,total_ratings,score,price
0,Counter-Strike,127873,0.97,7.19
1,Team Fortress Classic,3951,0.84,3.99
2,Day of Defeat,3814,0.90,3.99
3,Deathmatch Classic,1540,0.83,3.99
4,Half-Life: Opposing Force,5538,0.95,3.99
...,...,...,...,...
27070,Room of Pandora,3,1.00,2.09
27071,Cyber Gun,9,0.89,1.69
27072,Super Star Blast,1,0.00,3.99
27073,New Yankee 7: Deer Hunters,2,1.00,5.19


In [11]:
#The reason we're adding this is for tags with multiple words they must be connected
df['steamspy_tags'] = df['steamspy_tags'].str.replace(' ','-')
#TFIDF
df['genres'] = df['steamspy_tags'].str.replace(';',' ')
# count the number of occurences for each genre in the data set
counts = dict()
for i in df.index:
  #for each element in list (each row, split by ' ', in genres column) 
  #-- we're splitting by space so tfidf can interpret the cells
   for g in df.loc[i,'genres'].split(' '):
#if element is not in counts(dictionary of genres)
      if g not in counts:
        #give genre dictonary entry the value of 1
         counts[g] = 1
      else:
        #increase genre dictionary entry by 1
         counts[g] = counts[g] + 1

In [12]:
# create an object for TfidfVectorizer
tfidf_vector = TfidfVectorizer(stop_words='english')
# apply the object to the genres column
# convert the list of documents (rows of genre tags) into a matrix 
tfidf_matrix = tfidf_vector.fit_transform(df['genres'])

In [13]:
tfidf_matrix.shape #27075 games and 370 genres

(27075, 370)

In [14]:
# here we will create the cosine similiarity matrix
similarity_matrix = linear_kernel(tfidf_matrix,tfidf_matrix) 
print(similarity_matrix)

[[1.         1.         0.53785954 ... 0.16328518 0.         0.        ]
 [1.         1.         0.53785954 ... 0.16328518 0.         0.        ]
 [0.53785954 0.53785954 1.         ... 0.         0.         0.        ]
 ...
 [0.16328518 0.16328518 0.         ... 1.         0.61573349 0.61573349]
 [0.         0.         0.         ... 0.61573349 1.         1.        ]
 [0.         0.         0.         ... 0.61573349 1.         1.        ]]


In [37]:
# we'll create a function in order to find the closest name
def matching_score(a,b):
    return fuzz.ratio(a,b)

In [38]:
##These functions needed to return different attributes of the recommended game titles

#Convert index to title_year
def get_title_year_from_index(index):
   return df[df.index == index]['year'].values[0]
#Convert index to title
def get_title_from_index(index):
   return df[df.index == index]['name'].values[0]
#Convert index to title
def get_index_from_title(title):
   return df[df.name == title].index.values[0]
#Convert index to score
def get_score_from_index(index):
   return df[df.index == index]['score'].values[0]
#Convert index to total_ratings
def get_total_ratings_from_index(index):
   return df[df.index == index]['total_ratings'].values[0]
#Convert index to platform
def get_platform_from_index(index):
  return df[df.index == index]['platforms'].values[0]
#Convert index to price
def get_price_from_index(index):
  return df[df.index == index]['price'].values[0]
#Convert index to appid
def get_appid_from_index(index):
  return df[df.index == index]['appid'].values[0]
def get_publisher_from_index(index):
  return df[df.index == index]['publisher'].values[0]
   
# A function to return a title thats the closest to the user inputted title
# This helps with strictness so you don't have to be 100% accurate to get a result returned
def find_closest_title(title):
   leven_scores = list(enumerate(df['name'].apply(matching_score, b=title))) 
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True) 
   closest_title = get_title_from_index(sorted_leven_scores[0][0])
   distance_score = sorted_leven_scores[0][1]
   return closest_title, distance_score

In [39]:
#find_closest_title returns only one title, doing this we can get more, in this case 15; it can be any number 
def closest_names(title):
   leven_scores = list(enumerate(df['name'].apply(matching_score, b=title)))
   sorted_leven_scores = sorted(leven_scores, key=lambda x: x[1], reverse=True)
   top_closest_names = [get_title_from_index(i[0]) for i in sorted_leven_scores[:15]]  
   return top_closest_names

In [18]:
#we can now use closest_names to return the top 15 results closest to the typed in name we want

closest_names('Counter')

['Hunter',
 'Rencounter',
 'Coaster',
 'Conquer',
 'Montero',
 'Counter Spell',
 'CounterAttack',
 'Counter Fight',
 'Graze Counter',
 'Counter-Strike',
 'Contrast',
 'Card Hunter',
 'Crowntakers',
 'Loot Hunter',
 'Contract']

In [19]:
closest_names('Bejewled')

['Bejeweled® 3',
 'Bejeweled Twist',
 'Bleed',
 'Deepworld',
 'Teeworlds',
 'Levelhead',
 'BeeFender',
 'NewOld',
 'Beglitched',
 'Bee Aware!',
 'Besiege',
 'Keebles',
 'Bleed 2',
 'Settled',
 'Life Beetle']

In [20]:
closest_names('Call of Duty')

['Call of Duty®',
 'Call of Duty® 2',
 'Call of Duty®: WWII',
 'Call of Duty®: Ghosts',
 'Call of War',
 'Call of Juarez™',
 'Call of Duty®: Black Ops',
 'Call of Bitcoin',
 'Call of Cthulhu®',
 'Ball of Light',
 'Call of Duty: World at War',
 'Call of Duty®: Black Ops II',
 'Call of the Ocean',
 'Claws of Furry',
 'All Is Dust']

In [ ]:
def Content_Based_Reccomendation_System(Game_Name, amount_of_games, dropdown_choice, sort_option, min_year, platform, min_score):
  #Return closest game title match
  closest_title, distance_score = find_closest_title(dropdown_choice)
  #Create a Dataframe with these column headers
  recommended_df = pd.DataFrame(columns=['AppID','Game Title','Publisher', 'Year', 'Score', 'Price', 'Total Ratings'])
  #Make the closest title whichever dropdown choice the user has chosen
  closest_title = dropdown_choice
  #find the corresponding index of the game title
  games_index = get_index_from_title(closest_title)
  #return a list of the most similar game indexes as a list
  games_list = list(enumerate(similarity_matrix[int(games_index)]))
  #Sort list of similar games from top to bottom
  similar_games = list(filter(lambda x:x[0] != int(games_index), sorted(games_list,key=lambda x:x[1], reverse=True)))
  #Print the game title the similarity matrix is based on
  print('Here\'s the list of games similar to '+'\033[1m'+str(closest_title)+'\033[0m'+'.\n')
  #returns similar platform games
  n_games = []
  for i,s in similar_games:
    if platform in get_platform_from_index(i):
      n_games.append((i,s))
  # This will return anything above a minimum score provided
  high_scores = []
  for i,s in n_games:
    if get_score_from_index(i) > min_score:
      high_scores.append((i,s))
    
  #Return the game tuple (game index, game distance score) and store in a dataframe
  for i,s in n_games[:amount_of_games]: 
    #Dataframe will contain attributes based on game index
    row = {'AppID': get_appid_from_index(i),
           'Game Title': get_title_from_index(i), 
           'Publisher': get_publisher_from_index(i),
           'Year': get_title_year_from_index(i),
           'Score': get_score_from_index(i), 
           'Price': get_price_from_index(i), 
           'Total Ratings': get_total_ratings_from_index(i),}
    #Append each row to this dataframe       
    recommended_df = recommended_df.append(row, ignore_index = True)
  #Sort dataframe by Sort_Option provided by user
  recommended_df = recommended_df.sort_values(sort_option, ascending=False)
  #Only include games released same or after minimum year selected
  recommended_df = recommended_df[recommended_df['Year'] >= min_year]

  return recommended_df

#Create list of unique calendar years based on main df column
years_sorted = sorted(list(df['year'].unique()))
#Ask user for input
print("What game do you want to compare to?")

names = closest_names(input())
#Interface will include these buttons based on parameters in the function with a dataframe output
dropdown = gr.Interface(Content_Based_Reccomendation_System, ["text", gr.inputs.Slider(1, 15, step=int(1)), 
                                                            gr.inputs.Dropdown(names), 
                                                            gr.inputs.Radio(['AppID','Year','Score','Price','Total Ratings']),
                                                            gr.inputs.Slider(int(years_sorted[0]), int(years_sorted[-1]), step=int(1)),
                                                            gr.inputs.Radio(['windows','xbox','playstation','linux','mac']),
                                                            gr.inputs.Slider(0, 10, step=0.1)], "dataframe")
dropdown.launch(debug=True)

What game do you want to compare to?
Counter Strike
Running on local URL:  http://127.0.0.1:7860/

To create a public link, set `share=True` in `launch()`.


Here's the list of games similar to Counter-Strike: Source.

Here's the list of games similar to Counter-Strike: Source.

Here's the list of games similar to Counter-Strike: Source.

